<a href="https://colab.research.google.com/github/dlkt101101/AMATH-445/blob/main/AMATH445_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AMATH 445
## Assignment 1
Prepared by: Darren Alexander Lam Kin Teng (20977843)

In [5]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

Importing the dataset

In [6]:
data = pd.read_csv('/content/Cell viability and extrusion dataset V1.csv')

In [7]:
data.shape

(617, 51)

In [8]:
data.head()

,Reference,DOI,Final_Alginate_Conc_(%w/v),Final_Gelatin_Conc_(%w/v),Final_GelMA_Conc_(%w/v),Final_Hyaluronic_Acid_Conc_(%w/v),Final_MeHA_Conc_(%w/v),Final_NorHA_Conc_(%w/v),Final_Fibroin/Fibrinogen_Conc_(%w/v),Final_P127_Conc_(%w/v),Final_Collagen_Conc_(%w/v),Final_Chitosan_Conc_(%w/v),Final_CS-AEMA_Conc_(%w/v),Final_TCP_Conc_(%w/v),Final_Gellan_Conc_(%w/v),Final_Nano/Methycellulose_Conc_(%w/v),Final_PEGTA_Conc_(%w/v),Final_PEGMA_Conc_(%w/v),Final_PEGDA_Conc_(%w/v),Final_Agarose_Conc_(%w/v),CaCl2_Conc_(mM),NaCl2_Conc_(mM),BaCl2_Conc_(mM),SrCl2_Conc_(mM),Physical_Crosslinking_Durantion_(s),Photocrosslinking_Duration_(s),Extrusion_Pressure (kPa),Extrusion_Rate_Lengthwise_(mm/s),Extrusion_Rate_Volume-wise_(mL/s),Nozzle_Movement_Speed_(mm/s),Inner_Nozzle_Outer_Diameter_(µm),Outer_Nozzle_Inner_Diameter_(µm),Fiber_Diameter_(µm),Fiber_Spacing_(µm),Cell_Density_(cells/mL),Syringe_Temperature_(°C),Substrate_Temperature_(°C),Days_Observed,Cell_Culture_Medium_Used?,DI_Water_Used?,Precrosslinking_Solution_Used?,Saline_Solution_Used?,EtOH_Solution_Used?,Photoinitiator_Used?,Enzymatic_Crosslinker_Used?,Matrigel_Used?,Conical_or_Straight_Nozzle,Primary/Not_Primary,Viability_at_time_of_observation_(%),Acceptable_Viability_(Yes/No),Acceptable_Pressure_(Yes/No)
0,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,2.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0.0,0,0.0,100.0,0.0,0,0,NaN,0,199.9,16.67,NaN,NaN,0,210.0,430,300.0,250000.0,22.0,22.0,0.208,Y,N,N,N,N,N,N,N,S,Primary,96.0,Y,Y
1,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,2.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0.0,0,0.0,100.0,0.0,0,0,NaN,0,199.9,16.67,NaN,NaN,0,210.0,430,300.0,250000.0,22.0,22.0,1.000,Y,N,N,N,N,N,N,N,S,Primary,72.0,N,N
2,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,2.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0.0,0,0.0,100.0,0.0,0,0,NaN,0,199.9,16.67,NaN,15.0,0,210.0,430,300.0,250000.0,22.0,22.0,3.000,Y,N,N,N,N,N,N,N,S,Primary,80.0,Y,Y
3,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0,199.9,16.67,NaN,15.0,0,210.0,430,300.0,250000.0,4.0,22.0,0.208,Y,N,N,N,N,N,N,N,S,Primary,96.0,Y,Y
4,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0,199.9,16.67,NaN,15.0,0,610.0,1280,300.0,250000.0,4.0,22.0,0.208,Y,N,N,N,N,N,N,N,S,Primary,77.0,N,N
